In [46]:
import xgboost
import numpy as np

In [47]:
def to_int(char):
    return int.from_bytes(char.encode("utf8"),"little")

def int_index(char):
    return to_int(char)-to_int('A')

def single_feature_extractor(word):
    ret = np.zeros(int_index('z')+1)
    for char in word:
        ret[int_index(char)]+=1
    return ret

def feature_extractor(words):
    return words, np.vstack([single_feature_extractor(ww) for ww in words])

In [48]:
feature_extractor("Z")

('Z', array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.]]))

In [49]:
reference_characters=["bill", "tom", "tim", "dan"]
texts = [[ww] for ww in reference_characters]
texts[0]+=['amy', 'sandy', 'danny']
texts[1]+=['amy', 'sandy', 'sara']
texts[2]+=['amy', 'sandy', 'sara', "sally"]
texts[3]+=['amy', 'sandy']

In [50]:
Xs = [] # Feature vectors
Ys = [] # Binary as to if this feature is the target
G_inds = [] # Group IDs
last_ind = 0
for (group,(reference_name, raw_text)) in enumerate(zip(reference_characters, texts)):
    names, vectors = feature_extractor(raw_text)
    Ys.extend([(name == reference_name) for name in names])
    Xs.extend(vectors)
    first_ind = last_ind
    last_ind += len(names)
    G_inds.append(slice(first_ind,last_ind))
    
Xs = np.asarray(Xs) 
Ys = np.asarray(Ys)
    
G_inds



[slice(0, 4, None), slice(4, 8, None), slice(8, 13, None), slice(13, 16, None)]

From https://mathoverflow.net/a/51396/8800
$$H_s(x) = \begin{cases} \tfrac{1}{2}-x & x \le 0,\\\\
\tfrac{1}{2}(1-x)^2 & 0 < x < 1\\\\
0 & x \ge 1
\end{cases}$$

In [30]:
x>=1

NameError: name 'x' is not defined

In [31]:
x=np.asarray([-1,1,1])*np.asarray([-1,0.99,1])



In [32]:
#objective(y_true, y_pred) -> grad, hess relative to each sample point
def soft_hinge(y_true, y_pred):
    y_true = 2*(y_true-0.5)
    y_pred = 2*(y_pred-0.5)
    
    x=y_true*y_pred
    loss= np.select([x<=0, x>=1], [0.5-x, 0], 0.5*(1-x)**2)
    dLdYp = np.select([x<=0, x>=1], [-y_true, 0], -y_true*(1-x)) #-y_true+(y_true**2)y_pred
    d2LdYp2 = np.select([x<=0, x>=1], [0, 0], -y_true**2)
    
    #loss = np.maximum(0, 1-y_true*y_pred)
    #dLdYp = -y_pred # -y_true * (y_true*y_pred<1)
    #d2LdYp2 = np.zeros_like(y_pred)
    
    print("y_true=", y_true)
    print("y_pred=", y_pred)
    print("loss=",loss)
    print("dLdYp=",dLdYp)
    print("-----------------")
    
    return loss, dLdYp
    
def hinge(y_true, y_pred):
    y_true = 2*(y_true-0.5)
    y_pred = 2*(y_pred-0.5)
    
    loss = np.maximum(0, 1-y_true*y_pred)
    dLdYp = -y_true * (y_true*y_pred<1)
    d2LdYp2 = np.zeros_like(y_pred)
    
    grad = (y_true-y_pred)*loss
    hess = -y_pred*dLdYp
    
    print("y_true=", y_true)
    print("y_pred=", y_pred)
    #print("loss=",loss)
    #print("dLdYp=",dLdYp)
    print("-----------------")
    
    return grad, hess

In [97]:
from scipy import sparse
Xsparse = sparse.csc_matrix(Xs)


In [100]:
Xsparse.nonzero()

(array([ 0,  0,  0,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  4,  4,
         4,  5,  5,  5,  6,  6,  6,  6,  6,  7,  7,  7,  8,  8,  8,  9,  9,
         9, 10, 10, 10, 10, 10, 11, 11, 11, 12, 12, 12, 12, 13, 13, 13, 14,
        14, 14, 15, 15, 15, 15, 15], dtype=int32),
 array([33, 40, 43, 32, 44, 56, 32, 35, 45, 50, 56, 32, 35, 45, 56, 44, 46,
        51, 32, 44, 56, 32, 35, 45, 50, 56, 32, 49, 50, 40, 44, 51, 32, 44,
        56, 32, 35, 45, 50, 56, 32, 49, 50, 32, 43, 50, 56, 32, 35, 45, 32,
        44, 56, 32, 35, 45, 50, 56], dtype=int32))

In [102]:
Xsparse.data-=1

In [103]:
Xsparse.eliminate_zeros()
Xsparse.nonzero()

(array([ 0,  3,  7, 11, 12], dtype=int32),
 array([43, 45, 32, 32, 43], dtype=int32))

True

In [132]:
def relabel_grp_preds(y_true, y_pred):
    assert (y_true==1).sum()==1
    assert (y_true==0).sum()==len(y_true)-1
    
    true_inds = y_true>0.5 # Actually it is always 0 or 1, but force it to logical
    other_inds = np.logical_not(true_inds)
    
    true_score = y_pred[true_inds]
    mean_other_scores = y_pred[other_inds].mean()
    
    # Fake out the scores
    y_pred[true_inds] = true_score - mean_other_scores/2
    y_pred[other_inds] += true_score/(len(y_true))

def var_logregobj(y_true, y_pred):
    
    #oy_pred=y_pred.copy()
    for g_ind in G_inds:
        relabel_grp_preds(y_true[g_ind], y_pred[g_ind]) #Inplace
    
    #print(np.vstack([oy_pred, y_pred, y_true]).T)
    #print("------------")
    
    y_pred = 1.0 / (1.0 + np.exp(-y_pred))
    grad = y_pred-y_true
    hess = y_pred * (1.0 - y_true)
    return grad, hess
    
def logregobj(y_true, y_pred):
    y_pred = 1.0 / (1.0 + np.exp(-y_pred))
    grad = y_pred-y_true
    hess = y_pred * (1.0 - y_true)
    
    #print("y_true=", y_true)
    #print("y_pred=", y_pred)
    #print("loss=",grad)
    #print("dLdYp=",hess)
    #print("-----------------")
    return grad, hess
    
cls = xgboost.XGBClassifier(silent=False, objective=var_logregobj)
                            #objective=logregobj)

cls.fit(Xs,Ys)
np.vstack([cls.predict_proba(Xs, output_margin=False)[:,1], Ys]).T

array([[  6.44729435e-01,   1.00000000e+00],
       [  2.08023470e-04,   0.00000000e+00],
       [  2.08023470e-04,   0.00000000e+00],
       [  2.08023470e-04,   0.00000000e+00],
       [  6.44729435e-01,   1.00000000e+00],
       [  2.08023470e-04,   0.00000000e+00],
       [  2.08023470e-04,   0.00000000e+00],
       [  6.44729435e-01,   0.00000000e+00],
       [  6.44729435e-01,   1.00000000e+00],
       [  2.08023470e-04,   0.00000000e+00],
       [  2.08023470e-04,   0.00000000e+00],
       [  6.44729435e-01,   0.00000000e+00],
       [  2.08023470e-04,   0.00000000e+00],
       [  6.44729435e-01,   1.00000000e+00],
       [  2.08023470e-04,   0.00000000e+00],
       [  2.08023470e-04,   0.00000000e+00]])

In [34]:
cls = xgboost.XGBClassifier(silent=True)#,# objective=var_logregobj)
                            #objective=logregobj)

cls.fit(Xs,Ys)
np.vstack([cls.predict_proba(Xs)[:,1], Ys]).T

array([[ 0.58630711,  1.        ],
       [ 0.11137826,  0.        ],
       [ 0.05736404,  0.        ],
       [ 0.11137826,  0.        ],
       [ 0.58630711,  1.        ],
       [ 0.11137826,  0.        ],
       [ 0.05736404,  0.        ],
       [ 0.40762231,  0.        ],
       [ 0.58630711,  1.        ],
       [ 0.11137826,  0.        ],
       [ 0.05736404,  0.        ],
       [ 0.40762231,  0.        ],
       [ 0.05736404,  0.        ],
       [ 0.58630711,  1.        ],
       [ 0.11137826,  0.        ],
       [ 0.05736404,  0.        ]], dtype=float32)

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0])

In [129]:
#!/usr/bin/python
import numpy as np
import xgboost as xgb

# Excuse the hack to quickly represent my data
Xs = np.zeros((16,58))
Xs[[0,0,0,1,1,1,2,2,2,2,2,3,3,3,3,4,4,4,5,5,5,6,6,6,6,6,7,7,7,8,8,8,9,9,9,10,10,10,10,10,11,11,11,12,12,12,12,13,13,13,14,14,14,15,15,15,15,15],[33,40,43,32,44,56,32,35,45,50,56,32,35,45,56,44,46,51,32,44,56,32,35,45,50,56,32,49,50,40,44,51,32,44,56,32,35,45,50,56,32,49,50,32,43,50,56,32,35,45,32,44,56,32,35,45,50,56]]+=1
Xs[[0,3,7,11,12],[43,45,32,32,43]]+=1
Ys = np.array([1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0])

dtrain = xgb.DMatrix(Xs, Ys)

# note: for customized objective function, we leave objective as default
# note: what we are getting is margin value in prediction
# you must know what you are doing
param = {'max_depth': 2, 'eta': 1, 'silent': 1}

# user define objective function, given prediction, return gradient and second order gradient
# this is log likelihood loss
def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    preds = 1.0 / (1.0 + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1.0 - preds)
    return grad, hess


# training with customized objective, we can also do step by step training
# simply look at xgboost.py's implementation of train
bst = xgb.train(param, dtrain, obj=logregobj)
print("custom", bst.predict(dtrain))
param['objective'] = 'binary:logistic'
dst = xgb.train(param, dtrain)
print("default", dst.predict(dtrain, output_margin=False))

custom [ 0.63275987 -2.23188353 -3.39481163 -2.23188353  0.63275987 -2.23188353
 -3.39481163 -0.53016824  0.63275987 -2.23188353 -3.39481163 -0.53016824
 -3.39481163  0.63275987 -2.23188353 -3.39481163]
default [ 0.6506952   0.10160211  0.03324739  0.10160211  0.6506952   0.10160211
  0.03324739  0.3616243   0.6506952   0.10160211  0.03324739  0.3616243
  0.03324739  0.6506952   0.10160211  0.03324739]


In [125]:
(np.abs((bst.predict(dtrain)-dst.predict(dtrain, output_margin=True))))

array([ 0.01066357,  0.05233479,  0.02484512,  0.05233479,  0.01066357,
        0.05233479,  0.02484512,  0.03815299,  0.01066357,  0.05233479,
        0.02484512,  0.03815299,  0.02484512,  0.01066357,  0.05233479,
        0.02484512], dtype=float32)

In [133]:
import itertools as it

In [134]:
list(it.combinations(list("abc")))

TypeError: Required argument 'r' (pos 2) not found

In [142]:
" ".join(["a","b","c"], last="and")

TypeError: join() takes no keyword arguments

In [144]:
?str.join